<!--<badge>--><a href="https://colab.research.google.com/github/gradsflow/gradsflow/blob/main/examples/nbs/02-TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

In [ ]:
import os
import sys

os.chdir("../../")

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from flash.core.data.utils import download_data
from flash.text import TextClassificationData
from gradsflow import AutoTextClassifier

import ray

In [ ]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")

datamodule = TextClassificationData.from_csv(
    "review",
    "sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    batch_size=8,
)

In [ ]:
suggested_conf = dict(
    optimizer=["adam", "adamw"],
    lr=(4e-3, 1e-2),
)

model = AutoTextClassifier(
    datamodule,
    suggested_backbones=["prajjwal1/bert-tiny"],
    suggested_conf=suggested_conf,
    max_epochs=1,
    optimization_metric="val_accuracy",
    n_trials=2,
)

print("AutoTextClassifier initialised!")
trainer_config = {"accelerator":"auto", "devices":1}
model.hp_tune(gpu=0.5, trainer_config=trainer_config)

In [ ]:
model.analysis.dataframe()

In [ ]:
from flash import Trainer
trainer = Trainer(accelerator = "auto", devices=1)

In [ ]:
trainer.validate(model.model, datamodule=datamodule)